# Saco de palavras de 1-5 grams 

## Solução:
1. Criar uma lista de "stopwords" adequada para termos de "n" palavras;
2. Gerar uma lista de "tokens" removendo pontuação e as "stopwords";
3. Criar 5 grupos de ngrams, de 1 a 5 palavras;
4. Definir um limite de ocorrências mínimo;
5. Iterar entre os grupos, removendo os termos duplicados de ngrams menores, por exemplo:
    * Se houver um bigram (('sindicato', 'metalúrgico'), 5), os onegram (('sindicato',), 5) e (('metalúrgico',), 5) são removidos.
6. São somados os 5 grupos e retorna a lista de termos mais frequentes.

## Conclusão:
1. Quanto maior a base de discursos processados, melhor o resultado.

In [134]:
from nltk import word_tokenize
from nltk.util import ngrams
from nltk.corpus import stopwords as nltk_stopwords
from collections import Counter
from string import punctuation
import requests
import re

# Endereço da API dos discursos
BABEL_API_URL = 'https://babel.labhackercd.leg.br/api/v1/manifestations?manifestation_type__id=1'

# Pegando os 200 primeiros discursos da API do Babel
speeches = []
data = requests.get(BABEL_API_URL).json()['results']
for i in range(2, 50):
    data += requests.get(BABEL_API_URL + '&page=%s' % i).json()['results']
    
for speech in data:
    for attr in speech['attrs']:
        if attr['field'] == 'original':
            speeches.append(attr['value'])
            break

In [135]:
# Criando a lista de stopwords em português
EXTRA_STOPWORDS = [
    'sr.', 'nesse', 'deputados', '``', "''", 'empresa', 'trabalhadores', 'tema', 'brasil', 'brasileiro',
    'brasileira', 'sociedade', 'grandes', 'meios', 'principal', 'deputada', 'nesta', 'valor', 'reais',
    'representante', 'brasileiros', 'necessidade', 'quero', 'ser', 'geral', 'todo', 'toda', 'estar',
    'ter', 'parlamentares', 'região', 'forma', 'parte', 'disso', 'debate', 'entregar', 'nessa', 'nome',
    'vez', 'quer', 'primeira', 'soberania', 'justamente', 'ponto', 'presentes', 'faz', 'futuro', 'podem',
    'maneira', 'falar', 'interesses', 'caso', 'espaço', 'entrega', 'deste', 'mesma', 'negócio', 'existe',
    'avançar', 'ano', 'diz', 'próprios', 'criação', 'própria', 'dizendo', 'trazer', 'preocupação', 'ali',
    'ficar', 'desse', 'importância', 'senhor', 'queremos', 'poderia', 'ir', 'próprio', 'área', 'segundo',
    'acontece', 'sei', 'sabem', 'comum', 'mim', 'tratar', 'obrigação', 'falando', 'discurso', 'muitas',
    'acabou', 'falou', 'outro', 'capacidade', 'força','querem', 'significa', 'serviço', 'dados', 'tentar',
    'ninguém', 'gente', 'ideia', 'deputadas', 'dentro', 'fazendo', 'mão', 'época', 'uso', 'fiz', 'último',
    'nenhum', 'números', 'alguma', 'acreditamos', 'achamos', 'pagar', 'paga', 'conjunto', 'contentam',
    'lacunas', 'esperamos', 'digo', 'sente', 'logo', 'pesquisar', 'principais', 'mudar', 'sequer', 'pagou',
    'pagando', 'traz', 'desafio', 'concreto', 'atende', 'tendo', 'aberto', 'curto', 'recebe', 'receber',
    'décadas', 'minutos', 'horas', 'segundos', 'motivo', 'posso', 'dando', 'entra', 'volto', 'construir',
    'algumas', 'passar', 'muita', 'nisso', 'deveria', 'dá', 'construir', 'muito', 'longo', 'muitas', 'outras',
    'existem', 'conseguimos', 'precisamos', 'feita', 'mencionou', 'falei', 'cujo', 'hora', 'dizia', 'venha',
    'conseguindo', 'conseguir', 'votar', 'alguém', 'somente', 'todas', 'fizemos', 'citar', 'saiba', 'boa',
    'deveriam', 'acontecendo', 'algo', 'apresentei', 'sabemos', 'outra', 'junto', 'daqui', 'aconteceu',
    'haver', 'sinto', 'preciso', 'muitos', 'minimamente', 'amanhã', 'ontem', 'partir', 'objetivo', 'opinião',
    'vivemos', 'discutindo', 'agradecemos', 'utilizados', 'contra', 'dona', 'falta', 'possa', 'manhã', 'novas',
    'após', 'pessoas', 'fundamental', 'desses', 'devido', 'item', 'século', 'domingo', 'sábado',
    'presidente', ',', '.', '...', 'é', 'questão', 'art', 'ordem', 'v.exa', ':', 'governo', 'sr', 'agência',
    'aqui', 'vai', 'artigo', '§', 'neste', 'vamos', 'agora', "''", 'fazer', 'mesa', 'ainda', 'porque', 'trata',
    'estrutura', 'sobre', 'então', 'todos', 'obstrução', 'votação', 'presença', 'deputados', 'vou', 'brasil',
    'discutir', 'vigência', 'colocar', 'regimento', 'momento', ';', 'dois', 'dessa', 'medida', 'proposta',
    'casa', 'matéria', 'queria', 'assim', 'possamos', 'microfone', 'certeza', 'hoje', 'profissional', 'deixar',
    'provisória', 'ora', 'base', 'importante', 'veto', 'fala', '!', 'aumento', 'inciso', 'sra.', 'talvez',
    'cima', 'servir', 'nunca', 'dias', 'deus', 'dei', 'entendemos', 'chega', 'possam', 'entendo', 'poderá',
    'celeridade', 'tirar', 'mista', 'fechou-se', 'lado', 'lido', 'repassado', 'demais', 'venho', 'marcar',
    'xiii', 'diálogo', 'podemos', 'apenas', 'poder', 'efeitos', 'pode', 'acordo', 'solicitação',
    'reflexão', '?', 'ausência', 'aprovada', 'lideranças', 'dizer', 'portanto', 'peço', 'recolher',
    'prática', 'pois', 'milhões', 'bilhões', 'melhoria', 'atividade', 'claro', 'saber', 'dar',
    'avanço', 'condições', 'desastre', 'especialmente', 'exatamente', 'política', 'vezes', 'fazê-lo', 'têm',
    'derrubar', 'precisa', 'custo', 'necessária', 'cláusula', 'proposição', '-', 'palavra', 'tempo', 'segundos',
    'fez', 'necessário', 'zero', 'interesse', 'srs', 'sr', 'sras', 'sra', 'deputado', 'presidente', 'é', 'nº',
    's.a.', 'v.exa.', 'v.exa', '#', 'anos', 'º', 'exa', 'mesa', 'legislatura', 'sessão', 'maioria', 'seguinte',
    'mandato', 'bilhões', 'quilômetros', 'ª', 'parabéns', 'membros', 'convido', 'usual', 'biênio',
    'brasil', 'palavra', 'discussão', 'período', 'início', 'pronunciamento', 'suplente', 'atividade', 'ação',
    'ações', 'daqueles', 'diferenças', 'pasta', 'milhares', 'srªs', 'emenda', 'àqueles', 'tamanha', 'mês',
    'capaz', 'km', 'modelo', 'tarefas', 'colegas', 'programa', 'voz', 'pronunciamento',
    'casa', 'sessão', 'deliberativa', 'solene', 'ordinária', 'extraordinária', 'encaminhado', 'orador',
    'divulgar', 'deputado', 'parlamentar', 'projeto', 'proposta', 'requerimento', 'destaque',
    'veto', 'câmara', 'senado', 'congresso', 'país', 'estado', 'brasil', 'lei', 'novo', 'nova',
    'política', 'povo', 'voto', 'partido', 'liderança', 'bancada', 'bloco', 'líder', 'lider', 'frente',
    'governo', 'oposição', 'presença', 'presente', 'passado', 'ausência', 'ausencia', 'ausente', 'obstrução',
    'registrar', 'aprovar', 'rejeitar', 'rejeição', 'sabe', 'matéria', 'materia', 'questão', 'ordem', 'emenda',
    'sistema', 'processo', 'legislativo', 'plenário', 'pedir', 'peço', 'comissão', 'especial', 'permanente',
    'apresentar', 'encaminhar', 'encaminho', 'orientar', 'liberar', 'apoiar', 'situação', 'fato', 'revisão',
    'tempo', 'pauta', 'discutir', 'discussão', 'debater', 'retirar', 'atender', 'colegas', 'autor', 'texto',
    'medida', 'união', 'república', 'audiência', 'audiencia', 'público', 'publico', 'reunião', 'agradecer',
    'solicitar', 'assistir', 'contrário', 'favorável', 'pessoa', 'comemorar', 'ato', 'momento', 'diretora',
    'possível', 'atenção', 'agradeço', 'naquele', 'necessárias', 'presidenta', 'compromisso', 'geradas',
    'primeiro', 'simplesmente', 'ideal', 'argumento', 'i', 'válido', 'envolvidos', 'nesse', 'aspecto',
    'existentes', 'normativo', 'irá', 'nada', 'melhor', 'esperarmos', 'pouco', 'resolvermos', 'problema',
    'postura', 'faltas', 'declara', '%', 'dia', 'obrigado', 'agradeço', 'agradecido', 'população',
    'maior', 'cada', 'bem', 'mundo', 'desta', 'mil', 'sendo', 'outros', '$', '!', '@', '#', '&', '(', ')',
    'r', 'sempre', 'além', 'semana', 'relação', 'onde', 'meio', 'inclusive', 'lá', 'vem', 'menos', 'menor',
    'qualquer', 'desde', 'ontem', 'hoje', 'exemplos', 'exemplo', 'tão', 'fim', 'janeiro', 'fevereiro', 'março',
    'abril', 'maio', 'junho', 'julho', 'agosto', 'setembro', 'outubro', 'novembro', 'dezembro', 'alguns',
    'durante', 'gostaria', 'três', 'conta', 'feito', 'através', 'antes', 'depois', 'verdade', 'bom', 'quase',
    'setor', 'aí', 'disse', 'principalmente', 'final', 'vão', 'coisa', 'ver', 'sentido', 'vários',
    'nenhuma', 'quanto', 'infelizmente', 'felizmente', 'número', 'duas', 'dois', 'tanto', 'acho', 'achar',
    'enquanto', 'deve', 'apelo', 'papel', 'últimos', 'faço', 'fazer', 'garantir', 'garantia', 'fica',
    'obrigado..', 'assunto', 'sido', 'vir', 'incrementar', 'central', 'aproximado', 'aproximadamente',
    'hipotética', 'hipotese', 'hipótese', 'superiores', 'entende', 'pedido', 'oradora', 'tal', 'v.exas',
    'favor', 'vota', 'nº', 'srª', 'vista', 'sim', 'dito', 'tudo', 'obrigado', 'º', 'profundamente', 'custódio',
    'divulgado', 'características', 'perfeito', 'começarmos', 'nomes', 'amigo', 'possibilidade', 'mensagem',
    'come', 'parabenizar', 'começar', 'hs', 'atendimento', 'povos', '¯', 'ocorreu', 'entanto', 'diante',
    'defender', 'dr.', '“', '”', '•', 'v.', './', 'és', 'senhoras', 'senhores', 'tipo', 'várias', 'gerais',
    'quais', 'dessas', 'deu', 'havia', 'devem', 'enfim', 'apesar', 'passa', 'chegou', 'vêm', 'parece', 'u'
]

ONEGRAM_STOPWORDS = [
    'grande', 'nacional', 'são', 'e', 'de', 'das', 'dos', 'da', 'do', 'federal', 'cedo', 'urgência', 'equipe',
    'produtos', 'serviços', 'pequeno', 'total', 'podermos', 'consenso', 'popular', 'mérito', 'único', 'pública',
    'escolha', 'acesso', 'pilotos', 'trabalhar', 'ministério', 'países', 'combate', 'estados', 'vida', 'cidade',
    'municípios', 'histórico', 'defesa', 'município', 'prefeito', 'ii', 'santa', 'vereadora', 'centro',
    'prefeitura', 'governador', 'código', 'apoio', 'exercício', 'categoria', 'campo', 'kit',
    'ministro', 'social', 'recursos', 'direito', 'empresas', 'comunicação', 'democracia',
    'tribuna', 'história', 'respeito', 'luta', 'oportunidade', 'dinheiro', 'públicos',
    'civil', 'qualidade', 'políticas', 'sociais', 'registro', 'públicas', 'crescimento',
    'responsabilidade', 'participação', 'importantes', 'gestão', 'minas', 'cidades', 'lugar',
    'problemas', 'decisão', 'mulher', 'nobre', 'capital', 'aprovação', 'humanos', 'estadual',
    'internacional', 'senador', 'redução', 'realmente', 'realidade', 'plano', 'partidos',
    'conselho', 'posição', 'medidas', 'termos', 'divulgação', 'econômico', 'federais', 'fiscal',
    'emprego', 'maiores', 'rede', 'ruas', 'regional', 'continuar', 'profissionais', 'sob',
    'homens', 'político', 'atual', 'nação', 'meses', 'grupo', 'áreas', 'fundo', 'iniciativa',
    'executivo', 'cerca', 'cidadão', 'prazo', 'homem', 'trabalhador', 'órgãos', 'campanha',
    'controle', 'mínimo', 'mundial', 'dúvida', 'legislação', 'relatório', 'emendas', 'atividades',
    'razão', 'resultado', 'instituições', 'brasileiras', 'líderes', 'última', 'secretário', 'precisam',
    'criar', 'movimento', 'data', 'fazem', 'novos', 'casos', 'ambiente', 'administração', 'distrito',
    'pesquisa', 'relator', 'cumprimento', 'causa', 'informações', 'evento', 'aliás', 'superior',
    'filho', 'tarde', 'caminho', 'dificuldades', 'risco', 'publicação', 'sobretudo', 'coisas',
    'obrigada', 'santo', 'solicito', 'cargos', 'condição', 'próximo', 'secretaria', 'formação',
    'penal', 'forte', 'representa', 'aprovado', 'acima', 'políticos', 'setores', 'chegar',
    'espírito', 'prefeitos', 'grave', 'solução', 'governos', 'conhecimento', 'espero',
    'preço', 'mudança', 'instituto', 'tributária', 'amigos', 'levar', 'diversos',
    'municipal', 'dado', 'filhos', 'proteção', 'pagamento', 'federação', 'entidades',
    'br-', 'funcionários', 'média', 'organização', 'veículos', 'difícil', 'patrimônio',
    'marco', 'geração', 'diversas', 'honra', 'aumentar', 'movimentos', 'idade', 'passou',
    'inclusão', 'responsável', 'única', 'busca', 'questões', 'operação', 'participar',
    'pequenos', 'ajudar', 'regime', 'vítimas', 'pior', 'orgulho', 'unidos', 'embora',
    'forças', 'inteiro', 'modo', 'simples', 'programas', 'legislativa', 'caixa', 'leis',
    'passada', 'cidadãos', 'dignidade', 'associação', 'absolutamente', 'contribuição',
    'trata-se', 'esforço', 'representantes', 'fizeram', 'vereador', 'ficou', 'volta',
    'quadro', 'lembrar', 'concluir', 'votos', 'classe', 'atuação', 'médio', 'receita',
    'palavras', 'nível', 'encontro', 'milhão', 'diferente', 'local', 'estaduais',
    'constitucional', 'recurso', 'certamente', 'nossa'
]


# Método para limpar texto do discurso tirando as notas do taquigrafo
def clear_speech(text):
    text = re.sub(r'\([^)]*\)', '', text)
    text = re.sub(r'[OA] SRA?[\w\s.]+-', '', text)
    text = re.sub(r'PRONUNCIAMENTO[\sA-Z]+\s', '', text)
#     text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\s[\.\"]+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[Vv]\.[Ee][Xx][Aa]\.', 'v.exa', text)
    text = re.sub(r'[Aa][Rr][Tt]\.', 'art', text)
    text = re.sub(r'[Ss][Rr][Ss]?\.', 'sr', text)
    text = re.sub(r'[Ss][Rr][Aa][Ss]?\.', 'sr', text)
    text = re.sub(r'\d', '', text)
    return text.strip()


def get_tokens(speeches, extra_stopwords=None):
    """
    Função que retorna tokens de discursos removendo as "stopwords".
    Argumentos:
        speeches: Recebe uma lista de discursos.
        stopwords: Recebe uma lista de palavras a serem retiradas dos textos.
    Retorna:
        Uma lista palavras do discurso que não estão nas "stopwords".
    """
    special_stopwords = ['são', 'nossa']
    stopwords = nltk_stopwords.words('portuguese') + list(punctuation) + EXTRA_STOPWORDS
    stopwords = [word for word in stopwords if word not in special_stopwords]
    if extra_stopwords:
        stopwords += extra_stopwords
    tokens = []
    for text in speeches:
        text = clear_speech(text)
        tokens += [i for i in word_tokenize(text.lower(), language='portuguese') if i not in stopwords]
    
    return tokens


def ngrams_by_limit(tokens, n, limit):
    """
    Função que retorna uma lista de ngrams de acordo com os argumento passados.
    Argumentos:
        tokens: Recebe uma lista de tokens já processados pelo nltk.word_tokenize.
        n: Recebe o número de palavras que deseja dividir o ngram.
        limit: Recebe o limite mínimo de ocorrência.
    Retorna:
        Uma lista de ngrams com ocorrência maior que "limite" e com "n" palavras.
    """
    ngrams_count = Counter(ngrams(tokens, n)).most_common()
    result = [x for x in ngrams_count if x[1] >= limit]
    return result


def clean_tokens(tokens, fivegrams=[], quadgrams=[], trigrams=[], bigrams=[], extra_stopwords=None):
    """
    Função que retorna uma lista de tokens filtradas pelos argumentos passados.
    Argumentos:
        tokens: Recebe uma lista de tokens já processados pelo nltk.word_tokenize.
        fivegrams: Recebe uma lista de fivegrams.
        quadgrams: Recebe uma lista de quadgrams.
        trigrams: Recebe uma lista de trigramas.
        bigrams: Recebe uma lista de bigramas.
        extra_stopwords: Recebe uma lista de stopwords.
    Retorna:
        Uma lista de tokens removendo os n-gramas e stopwords passados nos argumentos.
    """
    
    if fivegrams:
        pos_fivegram = []
        for i in range(len(tokens)-4):
            for word1, word2, word3, word4, word5 in fivegrams:
                if (tokens[i] == word1 and tokens[i+1] == word2 and tokens[i+2] == word3 and
                    tokens[i+3] == word4 and tokens[i+4] == word5):
                    pos_fivegram.append(i)

        for pos in reversed(pos_fivegram):
            del tokens[pos:pos+5]
      
    if quadgrams:
        pos_quadgram = []
        for i in range(len(tokens)-3):
            for word1, word2, word3, word4 in quadgrams:
                if (tokens[i] == word1 and tokens[i+1] == word2 and tokens[i+2] == word3 and
                    tokens[i+3] == word4):
                    pos_quadgram.append(i)

        for pos in reversed(pos_quadgram):
            del tokens[pos:pos+4]

    if trigrams:
        pos_trigram = []
        for i in range(len(tokens)-2):
            for word1, word2, word3 in trigrams:
                if tokens[i] == word1 and tokens[i+1] == word2 and tokens[i+2] == word3:
                    pos_trigram.append(i)

        for pos in reversed(pos_trigram):
            del tokens[pos:pos+3]

    if bigrams:
        pos_bigram = []
        for i in range(len(tokens)-1):
            for word1, word2 in bigrams:
                if tokens[i] == word1 and tokens[i+1] == word2:
                    pos_bigram.append(i)

        for pos in reversed(pos_bigram):
            del tokens[pos:pos+2]
    
    if extra_stopwords:
        new_tokens = [token for token in tokens if token not in extra_stopwords]
    else:
        new_tokens = tokens

    return new_tokens

#Definimos os tokens
tokens = get_tokens(speeches)

# Determinamos o limite de ocorrências usado no algoritmo
limit = Counter(tokens).most_common(int(len(speeches) * 0.2))[-1][1]
if limit < 3:
    limit = 3

print("Limite mínimo de ocorrências: %s" % limit)

Limite mínimo de ocorrências: 75


In [136]:
# Obtendo termos com 5 palavras
fivegrams = ngrams_by_limit(tokens, 5, limit)
fivegrams

[]

In [137]:
# Obtendo termos com 4 palavras
stop_fivegrams = []

if fivegrams:
    stop_fivegrams = list(list(zip(*fivegrams))[0])

quadgram_tokens = clean_tokens(tokens, stop_fivegrams)
quadgrams = ngrams_by_limit(quadgram_tokens, 4, limit)
quadgrams

[]

In [138]:
# Obtendo termos com 3 palavras
stop_fivegrams = stop_quadgrams = []

if fivegrams:
    stop_fivegrams = list(list(zip(*fivegrams))[0])

if quadgrams:
    stop_quadgrams = list(list(zip(*quadgrams))[0])

trigram_tokens = clean_tokens(tokens, stop_fivegrams, stop_quadgrams)
trigrams = ngrams_by_limit(trigram_tokens, 3, limit)
trigrams

[(('supremo', 'tribunal', 'federal'), 129), (('rio', 'grande', 'sul'), 84)]

In [139]:
# Obtendo termos com 2 palavras
stop_fivegrams = stop_quadgrams = stop_trigrams = []

if fivegrams:
    stop_fivegrams = list(list(zip(*fivegrams))[0])
    
if quadgrams:
    stop_quadgrams = list(list(zip(*quadgrams))[0])

if trigrams:
    stop_trigrams = list(list(zip(*trigrams))[0])

bigram_tokens = clean_tokens(tokens, stop_fivegrams, stop_quadgrams, stop_trigrams)
bigrams = ngrams_by_limit(bigram_tokens, 2, limit)
bigrams

[(('michel', 'temer'), 202),
 (('dilma', 'rousseff'), 194),
 (('eduardo', 'cunha'), 166),
 (('são', 'paulo'), 145),
 (('polícia', 'federal'), 108),
 (('distrito', 'federal'), 107),
 (('segurança', 'pública'), 91),
 (('crime', 'responsabilidade'), 84),
 (('ex-presidente', 'lula'), 82),
 (('operação', 'lava-jato'), 78)]

In [142]:
# Obtendo termos com 1 palavras
stop_fivegrams = stop_quadgrams = stop_trigrams = stop_bigrams = []

if fivegrams:
    stop_fivegrams = list(list(zip(*fivegrams))[0])
    
if quadgrams:
    stop_quadgrams = list(list(zip(*quadgrams))[0])

if trigrams:
    stop_trigrams = list(list(zip(*trigrams))[0])

if bigrams:
    stop_bigrams = list(list(zip(*bigrams))[0])

onegram_tokens = clean_tokens(tokens, stop_fivegrams, stop_quadgrams, stop_trigrams, stop_bigrams,
                             ONEGRAM_STOPWORDS)
onegrams = ngrams_by_limit(onegram_tokens, 1, limit)
onegrams[:20] # Os 20 primeiros onegrams

[(('impeachment',), 521),
 (('golpe',), 497),
 (('saúde',), 461),
 (('dilma',), 458),
 (('pt',), 438),
 (('corrupção',), 334),
 (('lula',), 322),
 (('justiça',), 290),
 (('direitos',), 288),
 (('trabalho',), 283),
 (('constituição',), 259),
 (('educação',), 255),
 (('crise',), 253),
 (('mulheres',), 249),
 (('economia',), 191),
 (('rio',), 187),
 (('crime',), 182),
 (('econômica',), 177),
 (('desenvolvimento',), 176),
 (('família',), 165)]

In [141]:
result_tokens = onegrams + bigrams + trigrams + quadgrams + fivegrams
result_tokens.sort(key=lambda x: x[1], reverse=True)
result_tokens

[(('impeachment',), 521),
 (('golpe',), 497),
 (('saúde',), 461),
 (('dilma',), 458),
 (('pt',), 438),
 (('corrupção',), 334),
 (('lula',), 322),
 (('justiça',), 290),
 (('direitos',), 288),
 (('trabalho',), 283),
 (('constituição',), 259),
 (('educação',), 255),
 (('crise',), 253),
 (('mulheres',), 249),
 (('michel', 'temer'), 202),
 (('dilma', 'rousseff'), 194),
 (('economia',), 191),
 (('rio',), 187),
 (('crime',), 182),
 (('econômica',), 177),
 (('desenvolvimento',), 176),
 (('eduardo', 'cunha'), 166),
 (('família',), 165),
 (('contas',), 157),
 (('segurança',), 153),
 (('presidência',), 151),
 (('brasília',), 146),
 (('josé',), 146),
 (('são', 'paulo'), 145),
 (('petrobras',), 139),
 (('parlamento',), 135),
 (('reforma',), 132),
 (('violência',), 129),
 (('supremo', 'tribunal', 'federal'), 129),
 (('servidores',), 127),
 (('pmdb',), 124),
 (('construção',), 123),
 (('eleitoral',), 120),
 (('psdb',), 120),
 (('golpista',), 118),
 (('agricultura',), 117),
 (('imprensa',), 116),
 (('